In [1]:
from import_library import *

In [2]:
print("CPU count: {}".format(multiprocessing.cpu_count()))

CPU count: 24


In [5]:
data_type ='mix' #mat, csv, mix

# dataloader
clinical_data = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = './data/mdALFF matrix excel/' #raw_data_path = './data/image data/dFC matrix matlab/'
if data_type!='mix':
    next_path =f'mdALFF_{data_type}.csv' #next_path ='C0005FC_z.mat't'
else:
    pass
def loader_(data_path = None, data_type='mat' ,mat_dtype='FCM_mean'):
    if '.mat' in data_path:
        mat = scipy.io.loadmat(data_path)
        get_value = mat[mat_dtype ]
        return get_value.shape

    else:
        pd_tb = pd.read_csv(data_path)
        pd_patient_index = pd_tb[pd_tb.columns[0]]
        pd_patient_value = pd_tb[pd_tb.columns[1::]]
        return pd_patient_value

if data_type == 'mix':
    var_mean_stack = [loader_(data_path = os.path.join(raw_data_path,"mdALFF_var.csv")),loader_(data_path = os.path.join(raw_data_path,"mdALFF_mean.csv"))]
    patient_value = np.concatenate([np.array(var_mean_stack[0]),np.array(var_mean_stack[1])], axis=1)
else:
    patient_value = loader_(data_path = os.path.join(raw_data_path, next_path))
patient_value_index = clinical_data['subj']


print(patient_value.shape)

(84, 600)


In [4]:
# model list
def model_list(grid=False, seed=123):
    # svr_param = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5],'gamma' : ('auto','scale')}
    # SVR_ = SVR()
    
    xgbr_param = {'nthread':[4], 'objective':['reg:squarederror'], 'learning_rate': [.03, 0.05, .07], 'max_depth': [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31], 'min_child_weight': [4], 
                                    'subsample': [0.7], 'colsample_bytree': [0.7], 'n_estimators': [400, 450, 500, 550, 600, 660]}
    XGBR_ =XGBRegressor(random_state = seed)

    lgbmr_param =  {'num_leaves': [7, 14, 21], 'learning_rate': [0.05, 0.005], 'max_depth': [10, 15, 25], 
                                        'min_data_in_leaf':[10, 15, 25], 'feature_fraction': [0.6, 0.8, 0.9],'cat_smooth': [1,10, 15, 20, 35], 'verbose': [-1]}
    
    lgbmr_param =  {'learning_rate': [0.01, 0.05, 0.001,0.005], 'max_depth': [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31], 'verbose': [-1]}
    
    
    LGBMR_ = LGBMRegressor(random_state=seed, verbose=-1)

    # mlpr_param =  {'hidden_layer_sizes': [(25), (50), (100)]}


    mlpr_param =  {'hidden_layer_sizes': [(25), (50), (75),(100), (25, 50), (50,100), (25, 50, 75),(50, 75, 100)], 'learning_rate_init': [0.01, 0.03, 0.001, 0.003, 0.0001, 0.0003],
                                        'activation': ['relu', 'tanh'], 'solver': ['adam', 'lbfgs'], 'learning_rate' : ['constant', 'adaptive', 'invscaling']}

    MLPR_ = MLPRegressor(activation = 'relu', solver = 'adam', max_iter=20000, random_state=seed, early_stopping=True, validation_fraction=0.50, verbose=0)

    if grid ==True:
        # model_stack = {"SVR":[SVR_, svr_param], "XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
        model_stack = {"MLPR":[MLPR_, mlpr_param], "XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
        # model_stack = {"XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
    else:
        # model_stack = {"SVR":SVR_, "XBGR":XGBR_, "LGBMR": LGBMR_}
        # model_stack = {"XBGR":XGBR_, "LGBMR": LGBMR_}
        model_stack = {"MLPR":MLPR_, "XBGR":XGBR_, "LGBMR": LGBMR_}
    return model_stack

In [6]:
grid =True
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        # classification : AD8 [0-1, >=2]{0,1}, MOCA [<=23, >=24]{0,1}
        # outcome_name = 'MOCA客觀認知測驗分數[<=23,>=24]' #AD8主觀認知障礙[0-1,>=2], MOCA客觀認知測驗分數[<=23,>=24]
        model_stack = model_list(grid=grid)
        kfold = KFold(n_splits=4)
        for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
            for model in model_stack:
                if model in ['MLPR', 'XBGR', 'LGBMR']:
                    label_ = clinical_data[outcome_name]
                    print("Outcome: {}".format(outcome_name))
                    print("--Start Training {} model--".format(model))
                    if 'MOCA' in outcome_name:
                        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
                    elif 'AD8' in outcome_name:
                        label_s = [1 if i>=2 else 0 for i in list(label_)]
                        label_ = label_+1
                    X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
                    y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)
                    X_train, X_valid, _, _ = train_test_split(X_train, list(Y_label) , stratify=list(Y_label), test_size=0.5, random_state=123)
                    y_train, y_valid, _, _ = train_test_split(y_train, list(Y_label) , stratify=list(Y_label), test_size=0.5,random_state=123)

                    if grid==False:
                        if model in 'MLPR':
                            model = model_stack[model].fit(X_train, y_train)
                        else:
                            
                            model = model_stack[model].fit(np.array(X_train), np.array(y_train), eval_set=[[np.array(X_valid), np.array(y_valid)]], eval_metric="mape", verbose=False)
                        train_pred =  model.predict(X_train)
                        
                        test_pred =  model.predict(X_valid)
                    else:
                        GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold, n_jobs=-1, verbose=-1, refit = 'neg_mean_squared_error', scoring=['neg_mean_squared_error','neg_mean_absolute_percentage_error'],
                                                 return_train_score=True)
                        GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold, n_jobs=-1, verbose=-1, refit = 'neg_mean_absolute_percentage_error', scoring="neg_mean_absolute_percentage_error",
                                                 return_train_score=True)
                        GridModel.fit(np.array(X_train).astype(np.float16), np.array(y_train).astype(np.int8))
                        param_grid = model_stack[model][1]
                        # print("Grid Metric", GridModel.cv_results_.keys())
                        # for res_ in GridModel.cv_results_:
                        #     if 'test_score' in res_:
                        #         print(res_,GridModel.cv_results_[res_])
                        print("Scoring", GridModel.best_score_)
                        print('Best parameter : [{}]\n'.format(GridModel.best_params_))
                        train_pred =  GridModel.predict(X_train)
                        test_pred =  GridModel.predict(X_valid)

                    print("Valid integer pred result: ",np.array(test_pred).astype(np.int8))
                    print("Valid ground truth stacks: ",np.array(y_valid))
                    print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                    print('Valid MAE: ', round(metrics.mean_absolute_error(y_valid,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_valid,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_valid,test_pred),5))
                    # print('Test MAE: ', round(metrics.mean_absolute_error(y_test,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_test,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_test,test_pred),5))
                    print('----------------------------------------------------')
                print('\n')
                

Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training MLPR model--
Scoring -0.6803627859255585
Best parameter : [{'activation': 'tanh', 'hidden_layer_sizes': (25, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'lbfgs'}]

Valid integer pred result:  [1 1 2 1 3 1 2 1 2 1 0 1 2 2 1 1 1 4 3 2 4 1 1 2 1 1 1 2 2 0 2 0]
Valid ground truth stacks:  [8 5 4 1 4 4 1 1 1 1 7 1 5 1 1 1 3 2 6 3 3 1 6 1 3 7 5 1 2 1 2 1]
Train MAE:  0.00088  ||  MSE:  0.0  ||  MAPE:  0.00052
Valid MAE:  1.787  ||  MSE:  6.2802  ||  MAPE:  0.64141
----------------------------------------------------


Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training XBGR model--
Scoring -0.8852092770238718
Best parameter : [{'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 4, 'n_estimators': 450, 'nthread': 4, 'objective': 'reg:squarederror', 'subsample': 0.7}]

Valid integer pred result:  [3 2 3 2 2 2 3 2 3 1 2 2 3 3 3 2 2 4 2 3 3 2 1 1 2 2 2 3 2 3 2 1]
Valid ground truth stacks:  [8 5 4 

In [7]:
np.mean([-0.76256473, -0.92615425, -1.04265153])

-0.9104568366666667